# Load the cleaned data

In [1]:
import pandas as pd

In [2]:
grouped_dialogue_path = "D:/MACSS PROGRAM/30122/MACS-60000-2024-Winter/data/Arknights_plot/grouped_dialogues.csv"
grouped_df = pd.read_csv(grouped_dialogue_path)

In [3]:
grouped_df.sample(12)

,Story,Character,Dialogue
60665,act19side_Dorothy's Vision,Rhine Defense Section Employee,Officer? Where are you going?
15682,main_8_Roaring Flare,Rhodes Island Operator,Transmission went through!
66258,act21side_IL Siracusano,Bernardo,"You've got guts, and I like talking with men l..."
51094,act17side_Stultifera Navis,Amaia,I'm saying it's a possibility.
68714,act22side_What the Firelight Casts,Moran,Why would they control a civilian?
36506,act13side_Near Light,Bald Marcin,(Smiles silently) ......
40312,act14side_Break the Ice,Kjarr,...Pfft. Haha... Hahaha.
75454,act25side_Lone Trail,Kal'tsit,"...... For the past while, I investigated seve..."
53711,act17side_Stultifera Navis,Skadi,Below you!
25448,act12d0_The Great Chief Returns,Tomimi,"Come on, Gavial! You already came so close t..."


In [7]:
grouped_df[grouped_df['Character'] == 'non-character']

,Story,Character,Dialogue
0,main_0_Evil Time Part 1,non-character,It's been a long time since we've last seen...
456,main_10_Shatterpoint,non-character,"Londinium, Auchterigg Borough Inside The Shard"
484,main_10_Shatterpoint,non-character,「I'll return to the city's outer barracks soon...
494,main_10_Shatterpoint,non-character,"6:50 A.M. \ Overcast 527km out from Londinium,..."
531,main_10_Shatterpoint,non-character,7:25 A.M. \ Overcast Ten kilometers from Londi...
...,...,...,...
88467,act9mini_Pinus Sylvestris,non-character,The spokesman has been yelling at us for over ...
88476,act9mini_Pinus Sylvestris,non-character,I knew that he felt something for his target. ...
88478,act9mini_Pinus Sylvestris,non-character,"Put on the spot like that, I can't knee-jerk a..."
88480,act9mini_Pinus Sylvestris,non-character,"And then he leaves. And there's Centaurea, who..."


To instantiate an utterance object, we generally need the following information (all ids should be of type string):

- id: representing the unique id of the utterance.
- speaker: a ConvoKit speaker object representing the speaker giving the utterance.
- root: the id of the root utterance of the conversation.
- reply_to: id of the utterance this was a reply to.
- timestamp: timestamp of the utterance.
- text: text of the utterance.

I'm trying to convert my data on game plot (from a pandas dataframe) to an utterance df that can be make into corpus in ConvoKit.

My dataframe have 3 columns: 'Story' (name of the story where dialogues happened), 'Character', 'Dialogue'; also, you can consider the dataframe to be ordered in time series, if you read along the index in order, that's what you experienced in game.

Now, to instantiate an utterance object, we generally need the following information (all ids should be of type string):

- id: representing the unique id of the utterance. [we don't have this, need a way to generate one for each row]
- speaker: a ConvoKit speaker object representing the speaker giving the utterance. [this is the character, we can use the Character name for this]
- root: the id of the root utterance of the conversation. [I'll give my insight of denoting conversation below]
- reply_to: id of the utterance this was a reply to. [Let's put None for all rows now]
- timestamp: timestamp of the utterance. [We don't have it, probably we can have some numbers in increasing order for this for all rows since its in order]
- text: text of the utterance. [the Dialogue]

For conversation: I've denoted the 

In [11]:
import pandas as pd
from convokit import Corpus, Speaker, Utterance

# grouped_df have 'Story', 'Character', and 'Dialogue' columns

# Initialize variables for conversation tracking
current_conversation_id = None
previous_character = None

# List to hold the utterance objects
utterances = []

for idx, row in grouped_df.iterrows():
    # Generate a unique ID for the utterance
    utterance_id = f"utt_{idx}"
    
    # Check if the current row signifies a new conversation
    if row['Character'] == "non-character" or previous_character == "non-character":
        current_conversation_id = utterance_id

    # Create a Speaker object (for now, just using the name)
    speaker = Speaker(id=row['Character'])
    
    # Create an Utterance object
    utterance = Utterance(
        id=utterance_id,
        speaker=speaker,
        conversation_id=current_conversation_id,
        reply_to=None, 
        timestamp=idx,  # Using the index as a simple timestamp
        text=row['Dialogue'],
        meta={'story': row['Story']}
    )

    # Add the utterance to the list
    utterances.append(utterance)

    # Update the previous character for the next iteration
    previous_character = row['Character']

# Create the Corpus
corpus = Corpus(utterances=utterances)

No configuration file found at C:\Users\Admin/.convokit/config.yml; writing with contents: 
# Default Backend Parameters
db_host: localhost:27017
data_directory: ~/.convokit/saved-corpora
default_backend: mem


In [12]:
corpus.print_summary_stats()

Number of Speakers: 2031
Number of Utterances: 88493
Number of Conversations: 6405


In [ ]:
corpus